# Scraper for Question 1
Scraping necessary data to look at the impact of the opening kill on the win rate of a round and the different factors of an opening kill.

Like the last scrape, this data is taken from [DreamTeam](https://dreamteam.gg/). This data relates to the first kill of each round of each match. The statistics scraped include: time to first kill, perpetrator, victim, weapon used, and side.


## 3rd Scrape: First Kills (DreamTeam)
Firstly, import the necessary libraries for this notebook.

In [1]:
# Import libraries
import pandas as pd
import urllib
import urllib.request
from time import time, sleep
import re
from loguru import logger
import json
from pprint import pprint

Next, functions are written that will be used later for various purposes.

In [2]:
# Functions

# Uses urllib to retrieve webpage and scrape JSON data
def get_json(url):
    # Retrieve HTML
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"})
    page = urllib.request.urlopen(req).read()

    # Sleep to prevent overflowing website
    sleep(1)

    # Output of page returned is JSON, no need for BeautifulSoup
    data = json.loads(page)
    return data

# Build the URL needed for scraping
def get_url(MATCHCODE):
    url = "https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/" + MATCHCODE + "/entryfrags"
    return url
    
# Scrape the first kill details
def scrape_first_kills(matchID, MATCHCODE):
    
    df = pd.DataFrame(columns=["Match ID", "Round", "Team", "Player", "Side", "Time (Seconds)", "Weapon Used", "Victim", "Victim Weapon", "Won Round"])
    
    # Retrieve URL
    url = get_url(MATCHCODE)
    
    # Logger message
    logger.info('Scraping new match: %s' % url)
    
    # Get JSON
    data = get_json(url)
    
    # Get round count for iterating through first kills
    roundCount = data["total"]

    # Get details needed from each round and append to DataFrame
    for x in range(roundCount):
        roundNum = data["items"][x]["round"]
        teamName = data["items"][x]["player"]["team"]
        playerName = data["items"][x]["player"]["nickname"]
        side = data["items"][x]["player"]["side"]
        time = data["items"][x]["time"]
        weaponUsed = data["items"][x]["player"]["weapon"]
        victim = data["items"][x]["victim"]["nickname"]
        victimWeapon = data["items"][x]["victim"]["weapon"]

        df = df.append({"Match ID": matchID, "Round": roundNum, "Team": teamName, "Player": playerName,
                                   "Side": side, "Time (Seconds)": time, "Weapon Used": weaponUsed, "Victim": victim, "Victim Weapon": victimWeapon}, ignore_index=True)
        
    return df

The DTLinks.csv file that was cleaned and exported from the previous notebook is loaded into a DataFrame.

In [3]:
# Read in DreamTeam links from CSV file
dfDTLinks = pd.read_csv("DTLINKS.csv", index_col=0)
dfDTLinks.head(5)

,Event,Dream Team URL,Match ID,MATCHCODE
0,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,82050,5ca2589cbcf70a02726f6a24
1,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81796,5ca260136bd0f9026b37cb74
2,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81795,5ca260cd27b8d8026d731022
3,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81790,5ca260d3d35cfb023c7b2d94
4,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81761,5ca262c227b8d8026d731023


As we can see below, this DataFrame contains no null values.

In [4]:
print("-----------------------------------------")
print("DataFrame Info")
print(dfDTLinks.info())
print("-----------------------------------------\nDataFrame Null Values")
print(dfDTLinks.isnull().sum())
print("-----------------------------------------")

-----------------------------------------
DataFrame Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 428 entries, 0 to 427
Data columns (total 4 columns):
Event             428 non-null object
Dream Team URL    428 non-null object
Match ID          428 non-null int64
MATCHCODE         428 non-null object
dtypes: int64(1), object(3)
memory usage: 16.7+ KB
None
-----------------------------------------
DataFrame Null Values
Event             0
Dream Team URL    0
Match ID          0
MATCHCODE         0
dtype: int64
-----------------------------------------


Similar to the previous scrape, the DreamTeam calls the data to their page via an API that contains JSON data. This time the data contains info pertaining to the first kill of each round of a match. An example is printed below using PrettyPrint.

In [5]:
# Get test URL
testURL = get_url(dfDTLinks["MATCHCODE"].iloc[0])

# Get JSON data
data = get_json(testURL)

# Pretty Printer to show example of JSON output
pprint(data)

{'items': [{'headshot': True,
            'player': {'nickname': 'olofmeister',
                       'side': 'ct',
                       'steam_id': 'STEAM_1:1:14180732',
                       'team': 'FaZe Clan',
                       'weapon': 'usp_silencer'},
            'round': 1,
            'time': 47,
            'victim': {'nickname': 's1mple',
                       'side': 'terrorist',
                       'steam_id': 'STEAM_1:1:36968273',
                       'team': 'Natus Vincere',
                       'weapon': 'p250'}},
           {'headshot': True,
            'player': {'nickname': 'GuardiaN',
                       'side': 'ct',
                       'steam_id': 'STEAM_1:1:6032647',
                       'team': 'FaZe Clan',
                       'weapon': 'p250'},
            'round': 2,
            'time': 51,
            'victim': {'nickname': 's1mple',
                       'side': 'terrorist',
                       'steam_id': 'STEAM_1:1:36968273

Using a function written above, the JSON data is scraped for all necessary details. The corresponding Match ID is taken from dfDTLinks and appended.

In [6]:
dfFirstKills = pd.DataFrame(columns=["Match ID", "Round", "Team", "Player", "Side", "Time (Seconds)", "Weapon Used", 
                                     "Victim", "Victim Weapon", "Won Round"])

# Using the iterrows function, append the Match ID to the corresponding row in dfFirstKills
for _, row in dfDTLinks.iterrows():
    dfFirstKills = dfFirstKills.append(scrape_first_kills(row["Match ID"], row["MATCHCODE"]))
    
dfFirstKills

2019-05-10 20:00:25.748 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca2589cbcf70a02726f6a24/entryfrags


2019-05-10 20:00:27.278 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca260136bd0f9026b37cb74/entryfrags


2019-05-10 20:00:28.787 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca260cd27b8d8026d731022/entryfrags


2019-05-10 20:00:30.333 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca260d3d35cfb023c7b2d94/entryfrags


2019-05-10 20:00:31.884 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5ca262c227b8d8026d731023/entryfrags


...

2019-05-10 20:11:21.372 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa398cdbb10b05f37119b4/entryfrags


2019-05-10 20:11:22.872 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39bbd43602057e2452d7/entryfrags


2019-05-10 20:11:24.388 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39bf3cb88705cc46dd23/entryfrags


2019-05-10 20:11:25.932 | INFO     | __main__:scrape_first_kills:30 - Scraping new match: https://api.dreamteam.gg/analytics/csgo/game-analytics-proxy/stats/replays/5caa39c3d436020565490336/entryfrags


,Match ID,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round
0,82050,1,FaZe Clan,olofmeister,ct,47,usp_silencer,s1mple,p250,NaN
1,82050,2,FaZe Clan,GuardiaN,ct,51,p250,s1mple,deagle,NaN
2,82050,3,FaZe Clan,NiKo,ct,42,m4a1,flamie,deagle,NaN
3,82050,4,Natus Vincere,electronic,terrorist,81,ak47,NiKo,deagle,NaN
4,82050,5,Natus Vincere,flamie,terrorist,8,awp,GuardiaN,awp,NaN
5,82050,6,Natus Vincere,electronic,terrorist,8,ak47,NiKo,awp,NaN
6,82050,7,FaZe Clan,GuardiaN,ct,19,awp,flamie,ak47,NaN
7,82050,8,FaZe Clan,rain,ct,44,m4a1,s1mple,awp,NaN
8,82050,9,FaZe Clan,NiKo,ct,22,m4a1,s1mple,awp,NaN
9,82050,10,Natus Vincere,Edward,terrorist,68,ak47,olofmeister,deagle,NaN


## Cleaning the Scraped Data
The collected that is then checked for discrepancies.

In [7]:
print("-----------------------------------------")
print("DataFrame Info")
print(dfFirstKills.info())
print("-----------------------------------------\nDataFrame Null Values")
print(dfFirstKills.isnull().sum())
print("-----------------------------------------")

-----------------------------------------
DataFrame Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11177 entries, 0 to 22
Data columns (total 10 columns):
Match ID          11177 non-null object
Round             11177 non-null object
Team              11177 non-null object
Player            11177 non-null object
Side              11177 non-null object
Time (Seconds)    11177 non-null object
Weapon Used       11177 non-null object
Victim            11177 non-null object
Victim Weapon     11139 non-null object
Won Round         0 non-null object
dtypes: object(10)
memory usage: 960.5+ KB
None
-----------------------------------------
DataFrame Null Values
Match ID              0
Round                 0
Team                  0
Player                0
Side                  0
Time (Seconds)        0
Weapon Used           0
Victim                0
Victim Weapon        38
Won Round         11177
dtype: int64
-----------------------------------------


Again we see that the data collected contains null values. Similar to the previous data scraped, not all the JSON data is complete or correct. Furthermore, the data collected here must be compared with the previous data scraped from DreamTeam to see if they align.

The previosly scraped data is loaded into a DataFrame.

In [8]:
dfRoundWinner = pd.read_csv("ROUNDWINNER.csv", index_col=0)
dfRoundWinner

,Event,Match ID,Round,Winner,Winner Side,Match Winner
0,IEM Katowice 2019,82050,1,FaZe Clan,ct,Natus Vincere
1,IEM Katowice 2019,82050,2,FaZe Clan,ct,Natus Vincere
2,IEM Katowice 2019,82050,3,Natus Vincere,terrorist,Natus Vincere
3,IEM Katowice 2019,82050,4,Natus Vincere,terrorist,Natus Vincere
4,IEM Katowice 2019,82050,5,Natus Vincere,terrorist,Natus Vincere
5,IEM Katowice 2019,82050,6,Natus Vincere,terrorist,Natus Vincere
6,IEM Katowice 2019,82050,7,FaZe Clan,ct,Natus Vincere
7,IEM Katowice 2019,82050,8,FaZe Clan,ct,Natus Vincere
8,IEM Katowice 2019,82050,9,Natus Vincere,terrorist,Natus Vincere
9,IEM Katowice 2019,82050,10,Natus Vincere,terrorist,Natus Vincere


Copying my method previously used, a group by is used on both DataFrames. Again the DataFrames are grouped over Match ID and the values are counted.

In [9]:
dfFirstKillsCheck = dfFirstKills.groupby("Match ID").count()
dfFirstKillsCheck

,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round
Match ID,,,,,,,,,
17523,23,23,23,23,23,23,23,22,0
17524,17,17,17,17,17,17,17,17,0
17525,20,20,20,20,20,20,20,20,0
17526,18,18,18,18,18,18,18,18,0
17527,22,22,22,22,22,22,22,22,0
17528,21,21,21,21,21,21,21,21,0
17529,23,23,23,23,23,23,23,23,0
17530,23,23,23,23,23,23,23,23,0
17532,25,25,25,25,25,25,25,25,0


In [10]:
dfRoundWinnerCheck = dfRoundWinner.groupby("Match ID").count()
dfRoundWinnerCheck

,Event,Round,Winner,Winner Side,Match Winner
Match ID,,,,,
17523,23,23,23,23,23
17524,17,17,17,17,17
17525,20,20,20,20,20
17526,18,18,18,18,18
17527,22,22,22,22,22
17528,21,21,21,21,21
17529,23,23,23,23,23
17530,23,23,23,23,23
17532,25,25,25,25,25


A new DataFrame is created by concatenating the two most recent DataFrames for comparison.

In [11]:
dfFKRWCompare = pd.concat([dfFirstKillsCheck, dfRoundWinnerCheck], axis=1)
dfFKRWCompare

,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round,Event,Round,Winner,Winner Side,Match Winner
Match ID,,,,,,,,,,,,,,
17523,23,23,23,23,23,23,23,22,0,23,23,23,23,23
17524,17,17,17,17,17,17,17,17,0,17,17,17,17,17
17525,20,20,20,20,20,20,20,20,0,20,20,20,20,20
17526,18,18,18,18,18,18,18,18,0,18,18,18,18,18
17527,22,22,22,22,22,22,22,22,0,22,22,22,22,22
17528,21,21,21,21,21,21,21,21,0,21,21,21,21,21
17529,23,23,23,23,23,23,23,23,0,23,23,23,23,23
17530,23,23,23,23,23,23,23,23,0,23,23,23,23,23
17532,25,25,25,25,25,25,25,25,0,25,25,25,25,25


A mask is used to keep only the matches where the values of the two previous DataFrames match.

In [12]:
dfFKRWCompare = dfFKRWCompare[dfFKRWCompare["Player"]==dfFKRWCompare["Winner"]]
dfFKRWCompare

,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round,Event,Round,Winner,Winner Side,Match Winner
Match ID,,,,,,,,,,,,,,
17523,23,23,23,23,23,23,23,22,0,23,23,23,23,23
17524,17,17,17,17,17,17,17,17,0,17,17,17,17,17
17525,20,20,20,20,20,20,20,20,0,20,20,20,20,20
17526,18,18,18,18,18,18,18,18,0,18,18,18,18,18
17527,22,22,22,22,22,22,22,22,0,22,22,22,22,22
17528,21,21,21,21,21,21,21,21,0,21,21,21,21,21
17529,23,23,23,23,23,23,23,23,0,23,23,23,23,23
17530,23,23,23,23,23,23,23,23,0,23,23,23,23,23
17532,25,25,25,25,25,25,25,25,0,25,25,25,25,25


Once again, the clean Match IDs are taken and stored in a new DataFrame.

In [13]:
dfCleanIDs = pd.DataFrame(dfFKRWCompare.index.values, columns=["Match ID"])
dfCleanIDs

,Match ID
0,17523
1,17524
2,17525
3,17526
4,17527
5,17528
6,17529
7,17530
8,17532
9,17533


dfRoundWinner and dfCleanIDs are merged on Match ID, leaving only the valid matches.

In [14]:
dfRoundWinner["Match ID"] = pd.to_numeric(dfRoundWinner["Match ID"])

dfRoundWinnerClean = pd.merge(dfRoundWinner, dfCleanIDs, on=["Match ID"], how="inner")
dfRoundWinnerClean

,Event,Match ID,Round,Winner,Winner Side,Match Winner
0,IEM Katowice 2019,82050,1,FaZe Clan,ct,Natus Vincere
1,IEM Katowice 2019,82050,2,FaZe Clan,ct,Natus Vincere
2,IEM Katowice 2019,82050,3,Natus Vincere,terrorist,Natus Vincere
3,IEM Katowice 2019,82050,4,Natus Vincere,terrorist,Natus Vincere
4,IEM Katowice 2019,82050,5,Natus Vincere,terrorist,Natus Vincere
5,IEM Katowice 2019,82050,6,Natus Vincere,terrorist,Natus Vincere
6,IEM Katowice 2019,82050,7,FaZe Clan,ct,Natus Vincere
7,IEM Katowice 2019,82050,8,FaZe Clan,ct,Natus Vincere
8,IEM Katowice 2019,82050,9,Natus Vincere,terrorist,Natus Vincere
9,IEM Katowice 2019,82050,10,Natus Vincere,terrorist,Natus Vincere


The same is carried out on dfFirstKills.

In [15]:
dfFirstKills["Match ID"] = pd.to_numeric(dfFirstKills["Match ID"])

dfFirstKillsClean = pd.merge(dfFirstKills, dfCleanIDs, on=["Match ID"], how="inner")
dfFirstKillsClean

,Match ID,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round
0,82050,1,FaZe Clan,olofmeister,ct,47,usp_silencer,s1mple,p250,NaN
1,82050,2,FaZe Clan,GuardiaN,ct,51,p250,s1mple,deagle,NaN
2,82050,3,FaZe Clan,NiKo,ct,42,m4a1,flamie,deagle,NaN
3,82050,4,Natus Vincere,electronic,terrorist,81,ak47,NiKo,deagle,NaN
4,82050,5,Natus Vincere,flamie,terrorist,8,awp,GuardiaN,awp,NaN
5,82050,6,Natus Vincere,electronic,terrorist,8,ak47,NiKo,awp,NaN
6,82050,7,FaZe Clan,GuardiaN,ct,19,awp,flamie,ak47,NaN
7,82050,8,FaZe Clan,rain,ct,44,m4a1,s1mple,awp,NaN
8,82050,9,FaZe Clan,NiKo,ct,22,m4a1,s1mple,awp,NaN
9,82050,10,Natus Vincere,Edward,terrorist,68,ak47,olofmeister,deagle,NaN


A DataFrame containing only the winner of each round is created.

In [16]:
dfWonRound = pd.DataFrame(dfRoundWinnerClean["Winner"], columns=["Winner"])
dfWonRound

,Winner
0,FaZe Clan
1,FaZe Clan
2,Natus Vincere
3,Natus Vincere
4,Natus Vincere
5,Natus Vincere
6,FaZe Clan
7,FaZe Clan
8,Natus Vincere
9,Natus Vincere


This is then appended to dfFirstKillsClean. Now it will be easy to see if a team who got the first kill won the round.

In [17]:
dfFirstKillsClean = dfFirstKillsClean.join(dfWonRound)
dfFirstKillsClean

,Match ID,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round,Winner
0,82050,1,FaZe Clan,olofmeister,ct,47,usp_silencer,s1mple,p250,NaN,FaZe Clan
1,82050,2,FaZe Clan,GuardiaN,ct,51,p250,s1mple,deagle,NaN,FaZe Clan
2,82050,3,FaZe Clan,NiKo,ct,42,m4a1,flamie,deagle,NaN,Natus Vincere
3,82050,4,Natus Vincere,electronic,terrorist,81,ak47,NiKo,deagle,NaN,Natus Vincere
4,82050,5,Natus Vincere,flamie,terrorist,8,awp,GuardiaN,awp,NaN,Natus Vincere
5,82050,6,Natus Vincere,electronic,terrorist,8,ak47,NiKo,awp,NaN,Natus Vincere
6,82050,7,FaZe Clan,GuardiaN,ct,19,awp,flamie,ak47,NaN,FaZe Clan
7,82050,8,FaZe Clan,rain,ct,44,m4a1,s1mple,awp,NaN,FaZe Clan
8,82050,9,FaZe Clan,NiKo,ct,22,m4a1,s1mple,awp,NaN,Natus Vincere
9,82050,10,Natus Vincere,Edward,terrorist,68,ak47,olofmeister,deagle,NaN,Natus Vincere


A boolean column is added that tells whether the team that got the first kill won the round.

In [18]:
dfFirstKillsClean["Won Round"] = (dfFirstKillsClean["Team"]==dfFirstKillsClean["Winner"])
dfFirstKillsClean

,Match ID,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round,Winner
0,82050,1,FaZe Clan,olofmeister,ct,47,usp_silencer,s1mple,p250,True,FaZe Clan
1,82050,2,FaZe Clan,GuardiaN,ct,51,p250,s1mple,deagle,True,FaZe Clan
2,82050,3,FaZe Clan,NiKo,ct,42,m4a1,flamie,deagle,False,Natus Vincere
3,82050,4,Natus Vincere,electronic,terrorist,81,ak47,NiKo,deagle,True,Natus Vincere
4,82050,5,Natus Vincere,flamie,terrorist,8,awp,GuardiaN,awp,True,Natus Vincere
5,82050,6,Natus Vincere,electronic,terrorist,8,ak47,NiKo,awp,True,Natus Vincere
6,82050,7,FaZe Clan,GuardiaN,ct,19,awp,flamie,ak47,True,FaZe Clan
7,82050,8,FaZe Clan,rain,ct,44,m4a1,s1mple,awp,True,FaZe Clan
8,82050,9,FaZe Clan,NiKo,ct,22,m4a1,s1mple,awp,False,Natus Vincere
9,82050,10,Natus Vincere,Edward,terrorist,68,ak47,olofmeister,deagle,True,Natus Vincere


The data that was scraped from HLTV is then loaded in. This is so that new CSV files can be created from the HLTV data that only contains the same matches in the DreamTeam data.

In [19]:
dfHLTVMatches = pd.read_csv("MAJOR_MATCH_STATS.csv")
dfHLTVMatches

,Match ID,Event,Date,Team 1,Team 2,Map,Team 1 Rounds,Team 2 Rounds,Total Rounds,Winner,Team 1 FK,Team 2 FK,Team 1 Clutches,Team 2 Clutches,Team 1 Rating,Team 2 Rating
0,82145,IEM Katowice 2019,2019-03-03 18:05:00,ENCE,Astralis,Inferno,4,16,20,Astralis,11,9,1,0,0.61,1.56
1,82143,IEM Katowice 2019,2019-03-03 18:05:00,ENCE,Astralis,Train,11,16,27,Astralis,15,11,4,1,0.92,1.17
2,82123,IEM Katowice 2019,2019-03-02 19:20:00,MIBR,Astralis,Inferno,7,16,23,Astralis,9,14,0,4,0.90,1.25
3,82121,IEM Katowice 2019,2019-03-02 19:20:00,MIBR,Astralis,Overpass,14,16,30,Astralis,16,14,1,2,1.00,1.12
4,82120,IEM Katowice 2019,2019-03-02 15:10:00,Natus Vincere,ENCE,Mirage,14,16,30,ENCE,14,16,4,2,1.01,1.10
5,82119,IEM Katowice 2019,2019-03-02 15:10:00,Natus Vincere,ENCE,Dust2,16,3,19,Natus Vincere,15,4,1,0,1.65,0.44
6,82117,IEM Katowice 2019,2019-03-02 15:10:00,Natus Vincere,ENCE,Train,14,16,30,ENCE,14,16,2,1,0.93,1.19
7,82095,IEM Katowice 2019,2019-03-01 18:35:00,Astralis,NiP,Dust2,16,14,30,Astralis,19,11,1,0,1.07,0.97
8,82091,IEM Katowice 2019,2019-03-01 18:35:00,Astralis,NiP,Mirage,16,2,18,Astralis,13,5,1,1,1.58,0.49
9,82087,IEM Katowice 2019,2019-03-01 15:10:00,MIBR,Renegades,Train,16,12,28,MIBR,15,13,0,0,1.16,0.89


In [20]:
dfHLTVPlayers = pd.read_csv("MAJOR_PLAYER_STATS.csv")
dfHLTVPlayers

,Match ID,Event,Date,Player,Team,Map,Rounds,Kills,Deaths,FK,FD,FK%K,FD%D,Rating
0,82145,IEM Katowice 2019,2019-03-03 18:05:00,sergej,ENCE,Inferno,20,16,16,3,2,18.750000,12.500000,1.15
1,82145,IEM Katowice 2019,2019-03-03 18:05:00,xseveN,ENCE,Inferno,20,8,19,4,3,50.000000,15.789474,0.53
2,82145,IEM Katowice 2019,2019-03-03 18:05:00,allu,ENCE,Inferno,20,7,17,0,0,0.000000,0.000000,0.49
3,82145,IEM Katowice 2019,2019-03-03 18:05:00,Aerial,ENCE,Inferno,20,10,18,0,1,0.000000,5.555556,0.45
4,82145,IEM Katowice 2019,2019-03-03 18:05:00,Aleksib,ENCE,Inferno,20,6,19,4,3,66.666667,15.789474,0.45
5,82145,IEM Katowice 2019,2019-03-03 18:05:00,device,Astralis,Inferno,20,22,8,3,0,13.636364,0.000000,1.89
6,82145,IEM Katowice 2019,2019-03-03 18:05:00,dupreeh,Astralis,Inferno,20,22,9,4,2,18.181818,22.222222,1.80
7,82145,IEM Katowice 2019,2019-03-03 18:05:00,Xyp9x,Astralis,Inferno,20,16,8,1,0,6.250000,0.000000,1.51
8,82145,IEM Katowice 2019,2019-03-03 18:05:00,Magisk,Astralis,Inferno,20,15,11,0,5,0.000000,45.454545,1.37
9,82145,IEM Katowice 2019,2019-03-03 18:05:00,gla1ve,Astralis,Inferno,20,14,11,1,4,7.142857,36.363636,1.22


The two DataFrames are individually merged with dfCleanIDs, leaving only the valid matches.

In [21]:
dfHLTVMatchesClean = pd.merge(dfHLTVMatches, dfCleanIDs, on=["Match ID"], how="inner")
dfHLTVMatchesClean

,Match ID,Event,Date,Team 1,Team 2,Map,Team 1 Rounds,Team 2 Rounds,Total Rounds,Winner,Team 1 FK,Team 2 FK,Team 1 Clutches,Team 2 Clutches,Team 1 Rating,Team 2 Rating
0,82050,IEM Katowice 2019,2019-02-28 15:20:00,Natus Vincere,FaZe,Mirage,16,7,23,Natus Vincere,13,10,0,1,1.28,0.74
1,81790,IEM Katowice 2019,2019-02-22 12:00:00,Renegades,Vitality,Cache,11,16,27,Vitality,9,18,0,1,0.96,1.18
2,81761,IEM Katowice 2019,2019-02-21 18:30:00,Liquid,Natus Vincere,Dust2,16,9,25,Liquid,10,15,2,1,1.27,0.85
3,74061,FACEIT Major 2018,2018-09-22 19:05:00,Liquid,Astralis,Mirage,7,16,23,Astralis,9,14,0,2,0.82,1.23
4,74059,FACEIT Major 2018,2018-09-22 15:25:00,MIBR,Natus Vincere,Dust2,5,16,21,Natus Vincere,7,14,0,0,0.72,1.40
5,74055,FACEIT Major 2018,2018-09-21 19:35:00,Astralis,FaZe,Inferno,16,12,28,Astralis,13,15,1,2,1.13,0.96
6,74051,FACEIT Major 2018,2018-09-21 15:20:00,Liquid,HellRaisers,Cache,16,8,24,Liquid,11,13,2,1,1.32,0.80
7,74049,FACEIT Major 2018,2018-09-21 15:20:00,Liquid,HellRaisers,Dust2,10,16,26,HellRaisers,13,13,3,3,0.93,1.20
8,74026,FACEIT Major 2018,2018-09-20 17:05:00,compLexity,MIBR,Inferno,12,16,28,MIBR,8,20,2,1,0.78,1.27
9,74002,FACEIT Major 2018,2018-09-20 14:30:00,BIG,Natus Vincere,Dust2,2,16,18,Natus Vincere,6,12,0,2,0.50,1.58


In [22]:
dfHLTVPlayersClean = pd.merge(dfHLTVPlayers, dfCleanIDs, on=["Match ID"], how="inner")
dfHLTVPlayersClean

,Match ID,Event,Date,Player,Team,Map,Rounds,Kills,Deaths,FK,FD,FK%K,FD%D,Rating
0,82050,IEM Katowice 2019,2019-02-28 15:20:00,electronic,Natus Vincere,Mirage,23,24,12,4,0,16.666667,0.000000,1.75
1,82050,IEM Katowice 2019,2019-02-28 15:20:00,Zeus,Natus Vincere,Mirage,23,15,13,4,2,26.666667,15.384615,1.23
2,82050,IEM Katowice 2019,2019-02-28 15:20:00,flamie,Natus Vincere,Mirage,23,18,11,2,3,11.111111,27.272727,1.19
3,82050,IEM Katowice 2019,2019-02-28 15:20:00,Edward,Natus Vincere,Mirage,23,17,14,2,0,11.764706,0.000000,1.13
4,82050,IEM Katowice 2019,2019-02-28 15:20:00,s1mple,Natus Vincere,Mirage,23,15,12,1,5,6.666667,41.666667,1.11
5,82050,IEM Katowice 2019,2019-02-28 15:20:00,GuardiaN,FaZe,Mirage,23,14,18,3,3,21.428571,16.666667,0.87
6,82050,IEM Katowice 2019,2019-02-28 15:20:00,NiKo,FaZe,Mirage,23,15,18,3,2,20.000000,11.111111,0.86
7,82050,IEM Katowice 2019,2019-02-28 15:20:00,rain,FaZe,Mirage,23,14,18,1,4,7.142857,22.222222,0.73
8,82050,IEM Katowice 2019,2019-02-28 15:20:00,olofmeister,FaZe,Mirage,23,11,18,3,4,27.272727,22.222222,0.66
9,82050,IEM Katowice 2019,2019-02-28 15:20:00,AdreN,FaZe,Mirage,23,8,17,0,0,0.000000,0.000000,0.57


This cleaned HLTV data also allows for the map to be added to first kills DataFrame.

In [23]:
dfMaps = dfHLTVMatchesClean[["Match ID", "Map"]]
dfMaps

,Match ID,Map
0,82050,Mirage
1,81790,Cache
2,81761,Dust2
3,74061,Mirage
4,74059,Dust2
5,74055,Inferno
6,74051,Cache
7,74049,Dust2
8,74026,Inferno
9,74002,Dust2


The map column is added to the first kills DataFrame.

In [24]:
dfFirstKillsClean = pd.merge(dfFirstKillsClean, dfMaps, on=["Match ID"], how="inner")
dfFirstKillsClean

,Match ID,Round,Team,Player,Side,Time (Seconds),Weapon Used,Victim,Victim Weapon,Won Round,Winner,Map
0,82050,1,FaZe Clan,olofmeister,ct,47,usp_silencer,s1mple,p250,True,FaZe Clan,Mirage
1,82050,2,FaZe Clan,GuardiaN,ct,51,p250,s1mple,deagle,True,FaZe Clan,Mirage
2,82050,3,FaZe Clan,NiKo,ct,42,m4a1,flamie,deagle,False,Natus Vincere,Mirage
3,82050,4,Natus Vincere,electronic,terrorist,81,ak47,NiKo,deagle,True,Natus Vincere,Mirage
4,82050,5,Natus Vincere,flamie,terrorist,8,awp,GuardiaN,awp,True,Natus Vincere,Mirage
5,82050,6,Natus Vincere,electronic,terrorist,8,ak47,NiKo,awp,True,Natus Vincere,Mirage
6,82050,7,FaZe Clan,GuardiaN,ct,19,awp,flamie,ak47,True,FaZe Clan,Mirage
7,82050,8,FaZe Clan,rain,ct,44,m4a1,s1mple,awp,True,FaZe Clan,Mirage
8,82050,9,FaZe Clan,NiKo,ct,22,m4a1,s1mple,awp,False,Natus Vincere,Mirage
9,82050,10,Natus Vincere,Edward,terrorist,68,ak47,olofmeister,deagle,True,Natus Vincere,Mirage


The DreamTeam links DataFrame is once again cleaned, preventing unnecessary scraping in the future.

In [25]:
dfDTLinksClean = pd.merge(dfDTLinks, dfCleanIDs, on=["Match ID"], how="inner")
dfDTLinksClean

,Event,Dream Team URL,Match ID,MATCHCODE
0,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,82050,5ca2589cbcf70a02726f6a24
1,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81790,5ca260d3d35cfb023c7b2d94
2,IEM Katowice 2019,https://dreamteam.gg/csgo/analytics/match/5ca2...,81761,5ca262c227b8d8026d731023
3,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74061,5ca273f3d35cfb026f6f35e2
4,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74059,5ca274bbbcf70a0040573a99
5,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74055,5ca274d38da20e02aa25a612
6,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74051,5ca27568c0f32a02720a2203
7,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74049,5ca2760ebcf70a02726f6a2b
8,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74026,5ca276c6d43602024526e6b6
9,FACEIT London 2018,https://dreamteam.gg/csgo/analytics/match/5ca2...,74002,5ca276e2d43602027748b9e4


Cloud9 was renamed Cloud9 G2A for a period as part of a sponsorship deal. To combine the records, replace all occurences of Cloud9 G2A with Cloud9.

In [26]:
dfFirstKillsClean["Team"] = dfFirstKillsClean["Team"].replace({"Cloud9 G2A": "Cloud9"})

All the relevant DataFrames are then stored locally in CSV files.

In [27]:
dfFirstKillsClean.to_csv("FIRSTKILLS_CLEAN.csv", header=True, index=True)
dfDTLinksClean.to_csv("DTLINKS_CLEAN.csv", header=True, index=True)
dfRoundWinnerClean.to_csv("ROUNDWINNER_CLEAN.csv", header=True, index=True)
dfHLTVMatchesClean.to_csv("HLTVMATCHES_CLEAN.csv", header=True, index=True)
dfHLTVPlayersClean.to_csv("HLTVPLAYERS_CLEAN.csv", header=True, index=True)